In [5]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.special
import copy
import tensorflow as tf
import warnings
import time
import pandas as pd
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
class CNN:    
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learningrate):
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes
        self.lr = learningrate
        
        self.wih = np.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))
        
        self.lr = learningrate
        self.activation_function = lambda x: scipy.special.expit(x)
        
    def train(self, inputs_list, target_list):
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(target_list, ndmin=2).T
        
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, output_errors)
        
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0-final_outputs)), np.transpose(hidden_outputs))
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0-hidden_outputs)), np.transpose(inputs))
        
        pass
        
    def query(self, inputs_list):
        inputs = np.array(inputs_list, ndmin=2).T
        
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [12]:
input_nodes = 784
hidden_nodes = 200
output_nodes = 10
learning_rate = 0.2
cnn = CNN(input_nodes, hidden_nodes, output_nodes, learning_rate)

#Training Data
data_file = open("mnist_train.csv", 'r')
data_training_list = data_file.readlines()
data_file.close()

start = time.time()
epochs = 2
for e in range(epochs):
    for record in data_training_list:
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:])/255.0*0.99)+.01
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        cnn.train(inputs,targets)
        pass
    pass
end = time.time()
print("Time to completion: {}".format(end-start))

Time to completion: 188.00405263900757


In [13]:
test_data_file = open("mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

scorecard = []

for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    outputs = cnn.query(inputs)
    label = np.argmax(outputs)
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass    
    pass

scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.9662


In [62]:
#Tensorflow 
input_nodes = 784
hidden_nodes = 200
output_nodes = 10
learning_rate = 0.2
network_input= tf.placeholder(tf.float32, [None, input_nodes])
target_output = tf.placeholder(tf.float32, [None,output_nodes])

hidden_layer = tf.Variable(tf.random_normal([784, hidden_nodes]))
out_layer = tf.Variable(tf.random_normal([hidden_nodes, output_nodes]))

hidden_output = tf.nn.sigmoid(tf.matmul(network_input,hidden_layer))
total_output = tf.nn.sigmoid(tf.matmul(hidden_output,out_layer))

cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=total_output,labels=target_output))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

init = tf.global_variables_initializer()

In [76]:
csv = pd.read_csv("mnist_train.csv").values
sess = tf.Session()
sess.run(init)
start = time.time()
for epoch in range(2):
    for row in csv:
        inputs = (np.asfarray(row[1:]) / 255.0 * 0.99) + 0.01
        inputs = np.expand_dims(inputs, axis=0)
        label = np.zeros(output_nodes) + 0.01
        label[int(row[0])] = 0.99
        label = np.expand_dims(label, axis=0)
        features, label = inputs, label
        sess.run(training_step, feed_dict={network_input: features, target_output: label})
end = time.time()
print("Time to completion: {}".format(end-start)) 

Time to completion: 84.97453808784485


In [77]:
test_csv = pd.read_csv("mnist_test.csv").values
prediction = tf.argmax(total_output, 1)
num_correct = 0
for row in test_csv:
    inputs = (np.asfarray(row[1:])/255.0*0.99) + .01
    inputs = np.expand_dims(inputs, axis=0)
    label = int(row[0])
    label = np.expand_dims(label, axis=0)
    x = prediction.eval(feed_dict={network_input: inputs}, session=sess)
    if x[0] == label:
        num_correct += 1
total = num_correct/ len(test_csv)
print("performance = ", total)

performance =  0.933993399339934
